In [1]:
import geopandas as gpd
import pandas as pd
import os
import getpass
import matplotlib.pyplot as plt
from earthpy.clip import clip_shp

### NOTES:

###### Bring in necessary geojson files and set your projection for all files

In [5]:
wd = os.getcwd()
wd = wd[:wd.find('notebooks')]

#crs is set for Central Texas
crs =  {'init' :'epsg:6578'}

parks = gpd.read_file(wd+"data/coaparks/parkboundaries.geojson")
parks = parks.to_crs(crs).explode().reset_index()

quartbuff = gpd.read_file(wd+"data/coaparks_buffer/quarterbuffgj.geojson")
quartbuff = quartbuff.to_crs(crs).explode().reset_index()

pop = gpd.read_file(wd+"data/blockgroups_censusdata/popcount.geojson")
pop = pop.to_crs(crs)

race = gpd.read_file(wd+"data/blockgroups_censusdata/race.geojson")
race = race.to_crs(crs)

income = gpd.read_file(wd+"data/blockgroups_censusdata/income.geojson")
income = income.to_crs(crs)

#The race and pop and income files have already been exploded and reset in the 1)census notebook

##### Check your projections
https://geopandas.org/projections.html

In [6]:
parks.crs

{'init': 'epsg:6578'}

In [7]:
quartbuff.crs

{'init': 'epsg:6578'}

In [8]:
pop.crs

{'init': 'epsg:6578'}

In [9]:
race.crs

{'init': 'epsg:6578'}

###### Preview the files and clean the data

In [11]:
pop['fullarea_pop'] = pop['geometry'].area
pop.head()

,level_0,level_1,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,city_name,jurisdicti,Id,Id2,Total_POP,geometry,fullarea_pop
0,0,0,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,482090109011,2205,"POLYGON ((3069238.317597141 10028016.9612432, ...",3.246546e+06
1,0,1,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,482090109011,2205,"POLYGON ((3060340.445818828 10029833.26098835,...",1.955842e+07
2,1,0,482090109081,142299.0,0.0,+30.0960389,-097.8116716,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109081,482090109081,3774,"POLYGON ((3093548.488136804 10008631.07120929,...",1.837684e+03
3,2,0,484530001011,17128.0,0.0,+30.3353452,-097.7546582,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530001011,484530001011,1045,"POLYGON ((3111310.380106392 10089474.08471978,...",1.023445e+07
4,3,0,484530001012,43720.0,0.0,+30.3247732,-097.7516364,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530001012,484530001012,1202,"POLYGON ((3109356.370562974 10091521.99532213,...",1.190644e+07


In [12]:
race['fullarea_race'] = race['geometry'].area
race.head()

,level_0,level_1,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,city_name,jurisdicti,Id,...,Black or African American; Not Hispanic or Latino,American Indian and Alaska Native; Not Hispanic or Latino,Asian; Not Hispanic or Latino,Native Hawaiian and Other Pacific Islander; Not Hispanic or Latino,Some other race alone; Not Hispanic or Latino,Two or more races; Not Hispanic or Latino,Hispanic or Latino:,Sum of Non-Hispanic and Hispanic (Total Pop),geometry,fullarea_race
0,0,0,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,...,0,0,40,78,0,34,421,2205,"POLYGON ((3069238.317597141 10028016.9612432, ...",3.246546e+06
1,0,1,482090109011,291268.0,0.0,+30.1776997,-097.9267996,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109011,...,0,0,40,78,0,34,421,2205,"POLYGON ((3060340.445818828 10029833.26098835,...",1.955842e+07
2,1,0,482090109081,142299.0,0.0,+30.0960389,-097.8116716,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US482090109081,...,91,0,80,0,0,16,2507,3774,"POLYGON ((3093548.488136804 10008631.07120929,...",1.837684e+03
3,2,0,484530001011,17128.0,0.0,+30.3353452,-097.7546582,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530001011,...,26,0,0,0,0,20,113,1045,"POLYGON ((3111310.380106392 10089474.08471978,...",1.023445e+07
4,3,0,484530001012,43720.0,0.0,+30.3247732,-097.7516364,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530001012,...,0,14,0,0,0,42,121,1202,"POLYGON ((3109356.370562974 10091521.99532213,...",1.190644e+07


In [13]:
quartbuff['fullarea_buff'] = quartbuff['geometry'].area
quartbuff.head()

,level_0,level_1,ASSET_MGMT,LOCATION_N,ADDRESS,CITY_MUNIC,COUNTY,STATE,STREET_NUM,STREET_NAM,...,MXLOADID,GLOBALID,CREATED_BY,CREATED_DA,MODIFIED_B,MODIFIED_D,Shape__Are,Shape__Len,geometry,fullarea_buff
0,0,0,412,Stratford Overlook Greenbelt,"3006 Stratford Dr., Austin, Texas 78746",Austin,Travis,TX,3006,Stratford,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,4.403262e+03,420.672106,"POLYGON ((3099659.296639148 10076099.36654284,...",6.701295e+05
1,1,0,413,Highland Neighborhood Park,"403 W St Johns Ave., Austin, Texas 78752",Austin,Travis,TX,403,W St Johns,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.048800e+04,422.279483,"POLYGON ((3122902.889064377 10096179.90844941,...",6.708336e+05
2,2,0,354,Palma Plaza Pocket Park,"1524 Palma Plz., Austin, Texas 78703",Austin,Travis,TX,1524,Palma,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.007678e+04,492.661534,"POLYGON ((3109810.268547425 10075962.94345382,...",6.988492e+05
3,3,0,398,Oertli Neighborhood Park,"12613 Blaine Rd., Austin, Texas 78753",Austin,Travis,TX,12613,Blaine,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,2.671883e+05,2404.916487,"POLYGON ((3142020.189037737 10117140.78761685,...",1.463807e+06
4,4,0,370,Decker Tallgrass Prairie Preserve,"8001 Decker Ln., Austin, Texas 78724",Austin,Travis,TX,8001,Decker,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.515843e+07,26672.772067,"POLYGON ((3154723.006204471 10083882.87548318,...",9.294015e+06


In [14]:
quartbuff['fullarea_buff'] = quartbuff['geometry'].area
quartbuff.head()

,level_0,level_1,ASSET_MGMT,LOCATION_N,ADDRESS,CITY_MUNIC,COUNTY,STATE,STREET_NUM,STREET_NAM,...,MXLOADID,GLOBALID,CREATED_BY,CREATED_DA,MODIFIED_B,MODIFIED_D,Shape__Are,Shape__Len,geometry,fullarea_buff
0,0,0,412,Stratford Overlook Greenbelt,"3006 Stratford Dr., Austin, Texas 78746",Austin,Travis,TX,3006,Stratford,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,4.403262e+03,420.672106,"POLYGON ((3099659.296639148 10076099.36654284,...",6.701295e+05
1,1,0,413,Highland Neighborhood Park,"403 W St Johns Ave., Austin, Texas 78752",Austin,Travis,TX,403,W St Johns,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.048800e+04,422.279483,"POLYGON ((3122902.889064377 10096179.90844941,...",6.708336e+05
2,2,0,354,Palma Plaza Pocket Park,"1524 Palma Plz., Austin, Texas 78703",Austin,Travis,TX,1524,Palma,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.007678e+04,492.661534,"POLYGON ((3109810.268547425 10075962.94345382,...",6.988492e+05
3,3,0,398,Oertli Neighborhood Park,"12613 Blaine Rd., Austin, Texas 78753",Austin,Travis,TX,12613,Blaine,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,2.671883e+05,2404.916487,"POLYGON ((3142020.189037737 10117140.78761685,...",1.463807e+06
4,4,0,370,Decker Tallgrass Prairie Preserve,"8001 Decker Ln., Austin, Texas 78724",Austin,Travis,TX,8001,Decker,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,1.515843e+07,26672.772067,"POLYGON ((3154723.006204471 10083882.87548318,...",9.294015e+06


### Run spatial analysis on the amount of people distributed within a census block group against park buffer area

In [21]:
rp_intersection = gpd.overlay(race, quartbuff, how='intersection') #https://geopandas.org/set_operations.html
rp_intersection

,level_0_1,level_1_1,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,city_name,jurisdicti,Id,...,MXLOADID,GLOBALID,CREATED_BY,CREATED_DA,MODIFIED_B,MODIFIED_D,Shape__Are,Shape__Len,fullarea_buff,geometry
1,2,0,484530001011,17128.0,0.0,+30.3353452,-097.7546582,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530001011,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,3.937365e+06,(POLYGON ((3112577.691601671 10092106.43546111...
13,4,0,484530001013,415563.0,0.0,+30.3158528,-097.7515898,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530001013,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,3.937365e+06,"POLYGON ((3111467.239744052 10089394.70882012,..."
33,9,0,484530002042,16538.0,0.0,+30.3162020,-097.7451126,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530002042,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,3.937365e+06,"POLYGON ((3111980.956459695 10088537.07359926,..."
45,14,0,484530002061,34466.0,0.0,+30.3237904,-097.7443466,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530002061,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,3.937365e+06,(POLYGON ((3112469.330884764 10088913.07865507...
46,15,0,484530002062,61443.0,0.0,+30.3333205,-097.7472782,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484530002062,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,3.514350e+06,56641.286637,3.937365e+06,"POLYGON ((3113267.914431457 10091590.29026627,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,552,2,484910205081,636722.0,850.0,+30.4840639,-097.7595422,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205081,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,1.976420e+06,(POLYGON ((3104261.391998103 10149293.93935586...
2476,556,0,484910205101,2092.0,0.0,+30.4950296,-097.7673875,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205101,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,1.976420e+06,"POLYGON ((3105146.529438345 10149196.1606759, ..."
2477,557,0,484910205103,14153.0,0.0,+30.4953417,-097.7716305,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205103,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,6.289057e+05,3819.975953,1.976420e+06,(POLYGON ((3104692.752993851 10150897.72012676...
2474,554,0,484910205091,0.0,957.0,+30.5075259,-097.7591338,CITY OF AUSTIN,AUSTIN FULL PURPOSE,1500000US484910205091,...,None,None,CTM.Publisher,2019-10-12,CTM.Publisher,2019-10-12,2.473250e+06,15477.589042,3.951755e+06,"(POLYGON ((3107658.73525493 10158357.48293582,..."


race_quartbuff_inter=rp_intersection.copy().reset_index(drop=True, inplace=True)

race_quartbuff_inter["geometry"] = [MultiPolygon([feature]) if type(feature) == Polygon \
    else feature for feature in race_quartbuff_inter["geometry"]]

race_quartbuff_inter.to_file(wd+"data/access/randbuff_intersec_ACS17.geojson", driver='GeoJSON')

In [16]:
rp_intersection.columns
#

Index(['level_0_1', 'level_1_1', 'GEOID10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'city_name', 'jurisdicti', 'Id', 'Id2',
       'Total_POP', 'Not Hispanic or Latino:', 'White; Not Hispanic or Latino',
       'Black or African American; Not Hispanic or Latino',
       'American Indian and Alaska Native; Not Hispanic or Latino',
       'Asian; Not Hispanic or Latino',
       'Native Hawaiian and Other Pacific Islander; Not Hispanic or Latino',
       'Some other race alone; Not Hispanic or Latino',
       'Two or more races; Not Hispanic or Latino', 'Hispanic or Latino:',
       'Sum of Non-Hispanic and Hispanic (Total Pop)', 'fullarea_race',
       'level_0_2', 'level_1_2', 'ASSET_MGMT', 'LOCATION_N', 'ADDRESS',
       'CITY_MUNIC', 'COUNTY', 'STATE', 'STREET_NUM', 'STREET_NAM',
       'STREET_TYP', 'ZIPCODE', 'ASSET_SIZE', 'ACRE_SOURC', 'UNIT_OF_ME',
       'YEAR_ACQUI', 'ASSET_STAT', 'DEVELOPMEN', 'TPL_LANDUS', 'PARK_TYPE',
       'LEVEL_OF_S', 'LEVEL_OF_U', 'SERV

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
base = rp_intersection.plot(ax=ax, color="black", alpha=0.2)
layer = rp_intersection["geometry"].plot(ax=base, color='black', edgecolor = 'black')

In [17]:
rp_intersection[['GEOID10','Total_POP','fullarea_race','LOCATION_N','fullarea_buff','geometry']]

,GEOID10,Total_POP,fullarea_race,LOCATION_N,fullarea_buff,geometry
1,484530001011,1045,1.023445e+07,Shoal Creek Greenbelt,3.937365e+06,(POLYGON ((3112577.691601671 10092106.43546111...
13,484530001013,2016,1.151098e+07,Shoal Creek Greenbelt,3.937365e+06,"POLYGON ((3111467.239744052 10089394.70882012,..."
33,484530002042,1255,5.936942e+06,Shoal Creek Greenbelt,3.937365e+06,"POLYGON ((3111980.956459695 10088537.07359926,..."
45,484530002061,1252,6.393169e+06,Shoal Creek Greenbelt,3.937365e+06,(POLYGON ((3112469.330884764 10088913.07865507...
46,484530002062,884,5.914709e+06,Shoal Creek Greenbelt,3.937365e+06,"POLYGON ((3113267.914431457 10091590.29026627,..."
...,...,...,...,...,...,...
2472,484910205081,3775,3.664291e+07,Davis Spring Special Park,1.976420e+06,(POLYGON ((3104261.391998103 10149293.93935586...
2476,484910205101,4695,3.462268e+07,Davis Spring Special Park,1.976420e+06,"POLYGON ((3105146.529438345 10149196.1606759, ..."
2477,484910205103,929,3.727163e+06,Davis Spring Special Park,1.976420e+06,(POLYGON ((3104692.752993851 10150897.72012676...
2474,484910205091,2504,1.948094e+07,Brushy Creek Greenbelt,3.951755e+06,"(POLYGON ((3107658.73525493 10158357.48293582,..."


In [ ]:
race[['GEOID10','Total_POP','fullarea_race','geometry']]

In [ ]:
access_data = {'Park_Name':[],
       'Calc_Pop':[]}

pop_clip=rp_intersection.copy()

if park in rp_intersection:

        pop_clip['new_area'] = pop_clip['geometry'].area
        pop_clip['weight'] = pop_clip['new_area']/ pop_clip['full_area']
        pop_clip['new_pop'] = pop_clip['weight'] * pop_clip['Estimate;']

        data['Park_Name'].append(name)
        data['Calc_Pop'].append(pop_clip['new_pop'].sum())
    else: # if no pop blocks are within the park then return 0
        data['Park_Name'].append(name)
        data['Calc_Pop'].append(0)


In [ ]:
data = {'Park_Name':[],
       'Calc_Pop':[]}

from shapely.geometry import Polygon
names = quartbuff['LOCATION_N'].unique()
for i, dfrow in parks.iterrows():
     name = dfrow['LOCATION_N']

for i,name in enumerate(names):
    park = quartbuff.loc[quartbuff['LOCATION_N'] == name]
    park['geometry'] = park['geometry'].buffer(0) #This is the buffer
#     print(i, name)
    
    is_in_park = False
    for j, dfrow in pop.iterrows(): # iterate over pop shapes
#         print(dfrow['geometry'].is_valid)
        for n, dfrow2 in park.iterrows(): # iterate over buffered park shape
            # check if any pop shps fall within the buffered park shapes
            if Polygon(dfrow['geometry']).intersects(Polygon(dfrow2['geometry'])):
                is_in_park = True
                break
    if is_in_park:
        pop_valid = pop.copy()
        pop_valid['geometry'] = pop_valid['geometry'].buffer(0) # fix invalid polygons by buffering them by zero distances
        # definetly a hack and should be looked into.
        pop_clip = clip_shp(pop_valid,park)

        pop_clip['new_area'] = pop_clip['geometry'].area
        pop_clip['weight'] = pop_clip['new_area']/ pop_clip['full_area']
        pop_clip['new_pop'] = pop_clip['weight'] * pop_clip['Estimate;']

        data['Park_Name'].append(name)
        data['Calc_Pop'].append(pop_clip['new_pop'].sum())
    else: # if no pop blocks are within the park then return 0
        data['Park_Name'].append(name)
        data['Calc_Pop'].append(0)

# print(park.head())


In [ ]:
df = pd.DataFrame(data)
df['Calc_Pop'] = df['Calc_Pop'].astype(int)

df.head(10)

In [ ]:
parkspop = df.copy()
quartbuff = gpd.read_file(wd+"data/coaparks_buffer/quarterbuffgj.geojson")
qbuff = quartbuff.copy()
parksnocem= qbuff.loc[~qbuff['LOCATION_N'].str.contains('Cemetery')].reset_index()
parks_pop = parksnocem.merge(parkspop, left_on='LOCATION_N', right_on='Park_Name', suffixes=('_parksnocem', '_parkspop'))
parks_pop.to_file(wd+"data/access/parks_pop_quarterbuff_ACS17.shp")

In [ ]:
parks_pop

END OF CODE